In [2]:
from pyulog import *
from pyulog.px4 import *
from html import escape
import sqlite3
from IPython.core.display import display, HTML
import px4tools
from bs4 import BeautifulSoup as soup
from urllib.parse import urlparse, parse_qs
import os
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt

# importing sys
import sys
 
# adding FlightReview module to the system path
sys.path.insert(0, '../../flight_review/app/plot_app')

from helper import *
from db_entry import *
from configured_plots import generate_plots
from plotted_tables import *
from config_tables import *

BULK_PROCESS = False

In [3]:
# Helper methods
def read_ulg_data_as_df(file_name):
    data = None
    try:
        data = px4tools.read_ulog(file_name)
    except:
        None
    return data

def print_ulog_info(ulog):
    print('System: {:}'.format(ulog.msg_info_dict['sys_name']))
    if 'ver_hw' in ulog.msg_info_dict:
        print('Hardware: {:}'.format(ulog.msg_info_dict['ver_hw']))
    if 'ver_sw' in ulog.msg_info_dict:
        print('Software Version: {:}'.format(ulog.msg_info_dict['ver_sw']))
    # dropouts
    dropout_durations = [ dropout.duration for dropout in ulog.dropouts]
    if len(dropout_durations) > 0:
        total_duration = sum(dropout_durations) / 1000
        if total_duration > 5:
            total_duration_str = '{:.0f}'.format(total_duration)
        else:
            total_duration_str = '{:.2f}'.format(total_duration)
        print('Dropouts: {:} ({:} s)'.format(
            len(dropout_durations), total_duration_str))

    # logging duration
    m, s = divmod(int((ulog.last_timestamp - ulog.start_timestamp)/1e6), 60)
    h, m = divmod(m, 60)
    print('Logging duration: {:d}:{:02d}:{:02d}'.format( h, m, s))


In [35]:
# What the website actually plots
file_name = 'C:\\Users\\rwita\\Downloads\\87c1ccca-a4f4-4da0-94a1-a0ade11eded9.ulg'

ulog = ULog(file_name)
data = ulog.data_list
px4_ulog = PX4ULog(ulog)
px4_ulog.add_roll_pitch_yaw()
use_downsample = False
title = 'Flight Review - ' + px4_ulog.get_mav_type()

db_data = DBData()
vehicle_data = None

In [ ]:
print("message names: {:}".format(sorted([d.name for d in data])))
print()
print_ulog_info(ulog)

In [ ]:
info = {}
if ulog is not None:
    px4_ulog = PX4ULog(ulog)
    info['type'] = px4_ulog.get_mav_type()
    airframe_name_tuple = get_airframe_name(ulog)
    if airframe_name_tuple is not None:
        airframe_name, airframe_id = airframe_name_tuple
        if len(airframe_name) == 0:
            info['airframe'] = airframe_id
        else:
            info['airframe'] = airframe_name
    sys_hardware = ''
    if 'ver_hw' in ulog.msg_info_dict:
        sys_hardware = escape(ulog.msg_info_dict['ver_hw'])
        info['hardware'] = sys_hardware
    if 'sys_uuid' in ulog.msg_info_dict and sys_hardware != 'SITL':
        info['uuid'] = escape(ulog.msg_info_dict['sys_uuid'])
    branch_info = ''
    if 'ver_sw_branch' in ulog.msg_info_dict:
        branch_info = ' (branch: '+ulog.msg_info_dict['ver_sw_branch']+')'
    if 'ver_sw' in ulog.msg_info_dict:
        ver_sw = escape(ulog.msg_info_dict['ver_sw'])
        info['software'] = ver_sw + branch_info
        
info

In [ ]:
plots = generate_plots(ulog, px4_ulog, db_data, vehicle_data, "", "")

In [ ]:
tables = get_info_table_html(ulog, px4_ulog, db_data, vehicle_data, None)

In [ ]:
display(HTML(tables[0]))

In [ ]:
data = read_ulg_data_as_df(file_name)
messages = data.keys()
rc_messages = list(filter(lambda key: "input_rc" in key, messages))

In [ ]:
list(messages)

In [ ]:
data['t_manual_control_switches_0']

In [ ]:
# test.msg_info_dict
# test.msg_info_multiple_dict
# test.initial_parameters
# test.logged_messages
# get_total_flight_time(ulog)

In [ ]:
# Gathering one row worth of data
def get_row_of_data(log_link, ulog):
    row_data = {}
    row_data['link'] = log_link
    
    px4_ulog = PX4ULog(ulog)
        
    # GPS Present
    try:        
        gps_data = ulog.get_dataset('vehicle_gps_position')
        row_data['gps_data'] = "Yes"
    except:
        row_data['gps_data'] = "No"
        
    # Airframe
    row_data['airframe_type'] = px4_ulog.get_mav_type()
    row_data['airframe_name'] = get_airframe_name(ulog)[0]
    
    # Hardware
    row_data['hardware'] = ulog.msg_info_dict['ver_hw']
    
    # Software
    branch_info = ''
    if 'ver_sw_branch' in ulog.msg_info_dict:
        branch_info = ' (branch: ' + ulog.msg_info_dict['ver_sw_branch']+')'
    if 'ver_sw' in ulog.msg_info_dict:
        ver_sw = escape(ulog.msg_info_dict['ver_sw'])
        row_data['software'] = ver_sw + branch_info
        
    # Flight Duration        
    m, s = divmod(int((ulog.last_timestamp - ulog.start_timestamp)/1e6), 60)
    h, m = divmod(m, 60)
    row_data['flight_duration'] = '{:d}:{:02d}:{:02d}'.format(h, m, s)
    
    # Logging Start Time
    try:
        # get the first non-zero timestamp
        gps_data = ulog.get_dataset('vehicle_gps_position')
        indices = np.nonzero(gps_data.data['time_utc_usec'])
        if len(indices[0]) > 0:
            # we use the timestamp from the log and then convert it with JS to
            # display with local timezone.
            # In addition we add a tooltip to show the timezone from the log
            logging_start_time = int(gps_data.data['time_utc_usec'][indices[0][0]] / 1000000)

            utc_offset_min = ulog.initial_parameters.get('SDLOG_UTC_OFFSET', 0)
            utctimestamp = datetime.datetime.utcfromtimestamp(
                logging_start_time+utc_offset_min*60).replace(tzinfo=datetime.timezone.utc)

            row_data['start_time'] = str(utctimestamp.strftime('%d-%m-%Y %H:%M'))
    except:
        # Ignore. Eg. if topic not found
        pass
        
    # Flight Modes
    flight_mode_changes = get_flight_mode_changes(ulog)
    flight_mode_changes = filter(lambda elem: elem[1] != -1, flight_mode_changes)
    flight_mode_names = map(lambda elem: flight_modes_table[elem[1]][0], flight_mode_changes)
    row_data['flight_modes'] = list(set(flight_mode_names))
        
    # Altitude Information
    local_pos = ulog.get_dataset('vehicle_local_position')
    pos_z = local_pos.data['z']
    row_data['altitude_min'] = np.amin(pos_z)
    row_data['altitude_max'] = np.amax(pos_z)
    row_data['altitude_avg'] = np.mean(pos_z)
    
    # Parameters
    row_data['parameters'] = ulog.initial_parameters
    
    # Terrain Following
    if row_data['parameters']['MPC_ALT_MODE'] in [1, 2]:
        row_data['terrain_following'] = "Yes"
    else:
        row_data['terrain_following'] = "No"
        
    # Terrain Following
    if row_data['parameters']['COM_OBS_AVOID'] == 1:
        row_data['object_avoidance'] = "Yes"
    else:
        row_data['object_avoidance'] = "No"
    
    # Stuff that I did not find
    row_data['upload_date'] = ""
    row_data['description'] = ""
    row_data['gyroscope'] = ""
    row_data['mag_accel'] = ""
    row_data['barometer'] = ""
    row_data['compass'] = ""
    row_data['rating'] = ""
    row_data['error'] = ""
    row_data['soft_release_date'] = ""
    row_data['remote_control'] = ""
    row_data['waypoints'] = ""
    
    # Returning one row of data
    return row_data


# get_row_of_data("link", ulog)

In [ ]:
# Reading logs and extracting fields
if BULK_PROCESS:
    f = open('E:\\Purdue\\Spring 2023\\CS 590\\Work\\GitHub Issue Scraper\\scrape_data\\px4_log_links.json')
    data = json.load(f)
    results = []
    failed = []

    for key, value in tqdm(data.items()):
        parsed_issue_link = urlparse(key)
        issue_id = parsed_issue_link.path.split("/")[-1]
        if value['count'] > 0:
            for log_link in value['log_links']:
                parsed_log_link = urlparse(log_link)
                log_id = parse_qs(parsed_log_link.query)
                if 'log' in log_id:
                    log_id = parse_qs(parsed_log_link.query)['log'][0]
                    log_file_path = f"E:\\Purdue\\Spring 2023\\CS 590\\Work\\GitHub Issue Scraper\\Logs\\{issue_id}\\{log_id}.ulg"
                    if os.path.isfile(log_file_path):
                        try:
                            ulog = load_ulog_file(log_file_path)
                            results.append(get_row_of_data(log_link, ulog))
                        except:
                            failed.append(log_file_path)

    print(f"{len(results)} processed, {len(failed)} failed")

    df = pd.DataFrame.from_dict(results)
    new_df = df[['link', 'upload_date', 'gps_data', 'description', 'airframe_type', 'airframe_name', 'hardware', 'gyroscope',
           'mag_accel', 'barometer', 'compass', 'software', 'flight_duration', 'start_time', 'rating', 'error',
           'flight_modes', 'soft_release_date', 'remote_control', 'altitude_min', 'altitude_avg', 'altitude_max', 'waypoints', 'terrain_following',
           'object_avoidance', 'parameters']]
    new_df.to_excel("./parsed_df.xlsx")

In [ ]:
# # New
ulog.get_dataset('manual_control_switches')

# # Old
# ulog.get_dataset('manual_control_setpoint')

# Older
# ulog.get_dataset('rc_channels')

In [ ]:
df = pd.DataFrame(ulog.get_dataset('manual_control_setpoint').data)
df = df[['y', 'x', 'r', 'z', 'aux1', 'aux2']]
rc_used = False
for (i, value) in df.std().items():
    if value != 0:
        rc_used = True
        break
rc_used

In [ ]:
df.plot()
plt.show()

In [ ]:
df_data = ulog.get_dataset('manual_control_switches').data
df = pd.DataFrame(df_data)
df = df[['mode_slot', 'kill_switch']]
df['kill_switch'] = df['kill_switch'] == 1
flight_modes = list(df['mode_slot'].unique())
kill_switch_engaged = True in df['kill_switch'].unique()
print(f"Flight Modes: {flight_modes}")
print(f"Killswitch Engaged: {kill_switch_engaged}")

In [ ]:
px4_ulog.get_configured_rc_input_names(-1)

In [ ]:
ulog = load_ulog_file("7fa41e74-e051-4624-a891-97f72d663e55.ulg")
px4_ulog = PX4ULog(ulog)
df_data = ulog.get_dataset('rc_channels').data
df = pd.DataFrame(df_data)

num_rc_channels = min(8, np.amax(df_data['channel_count']))
channel_cols = ['channels['+str(i)+']' for i in range(num_rc_channels)]
channel_names = [px4_ulog.get_configured_rc_input_names(i) for i in range(num_rc_channels)]
df = df[channel_cols]

rc_used = False
for (i, value) in df.std().items():
    if value != 0:
        rc_used = True
        break
rc_used

In [ ]:
df[['channels['+str(i)+']' for i in range(num_rc_channels)]]

In [36]:
# console messages, perf & top output
top_data = {}
perf_data = {}
console_messages = []
if 'boot_console_output' in ulog.msg_info_multiple_dict:
    console_output = ulog.msg_info_multiple_dict['boot_console_output'][0]
    console_output = ''.join(console_output)
    console_messages = console_output.split("\n")

for state in ['pre', 'post']:
    if 'perf_top_'+state+'flight' in ulog.msg_info_multiple_dict:
        current_top_data = ulog.msg_info_multiple_dict['perf_top_'+state+'flight'][0]
        flight_data = escape('\n'.join(current_top_data))
        top_data['perf_top_'+state+'flight'] = current_top_data
    if 'perf_counter_'+state+'flight' in ulog.msg_info_multiple_dict:
        current_perf_data = ulog.msg_info_multiple_dict['perf_counter_'+state+'flight'][0]
        flight_data = escape('\n'.join(current_perf_data))
        perf_data['perf_counter_'+state+'flight'] = current_perf_data
if 'perf_top_watchdog' in ulog.msg_info_multiple_dict:
    current_top_data = ulog.msg_info_multiple_dict['perf_top_watchdog'][0]
    flight_data = escape('\n'.join(current_top_data))
    top_data['perf_top_watchdog'] = current_top_data

In [39]:
console_messages

['HW arch: PX4_FMU_V6C',
 'HW type: V6C000000',
 'HW version: 0x000',
 'HW revision: 0x000',
 'PX4 git-hash: 96362bfb52622a8b7a13423a28ff692d918fdce7',
 'PX4 version: 1.14.0 80 (17694848)',
 'OS: NuttX',
 'OS version: Release 11.0.0 (184549631)',
 'OS git-hash: 1ee8fb827c570f302a43ca9870c58d1ac73997d3',
 'Build datetime: Dec 14 2022 13:13:38',
 'Build uri: localhost',
 'Build variant: default',
 'Toolchain: GNU GCC, 9.3.1 20200408 (release)',
 'PX4GUID: 0006000000003235343331315103003a0036',
 'MCU: STM32H7[4|5]xxx, rev. V',
 'INFO  [param] selected parameter default file /fs/mtd_params',
 "INFO  [param] importing from '/fs/mtd_params'",
 'INFO  [parameters] BSON document size 2031 bytes, decoded 2031 bytes (INT32:40, FLOAT:61)',
 'INFO  [param] selected parameter backup file /fs/microsd/parameters_backup.bson',
 'Board architecture defaults: /etc/init.d/rc.board_arch_defaults',
 'Board defaults: /etc/init.d/rc.board_defaults',
 'Loading airframe: /etc/init.d/airframes/4019_x500_v2',
 "

In [42]:
"; ".join([f"<{message}>" for message in console_messages if "INFO  [gps]" in message])

'<INFO  [gps] u-blox firmware version: SPG 3.01>; <INFO  [gps] u-blox protocol version: 18.00>; <INFO  [gps] u-blox module: NEO-M8N-0>'